In [1]:
import math 
import string 
import re

import json 

import pandas as pd 
import sqlite3


In [2]:

def connect_to_db(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    return conn, c 

In [60]:
parent_path = '/home/ramprasad.sa'
db_path = f'{parent_path}/human_annotations_factuality/billsum/set1/annotated/billsum_summaries_set1_final.db'
conn, c = connect_to_db(db_path)

In [61]:
qualified_annotators_billsum = ['ann_krcnbm', 'ann_hguilf']
qualified_annotators_news = ['ann_japq', 'ann_tpfo']
qualified_annotators_pubmed = ['ann_jclvzw', 'ann_eftpco']
df_annotations = pd.read_sql('SELECT * from label', conn)
# df_annotations = df_annotations[df_annotations['user_id'].isin(qualified_annotators_billsum)]

# df_annotations

In [62]:
list(set(df_annotations['user_id']))

['ann_krcnbm', 'sanjana', 'ann_hguilf', 'ann_ckorrj', 'ann_dnifqq']

In [63]:

    
def remove_duplicates(df, qualified_annotators):
    processed_rows = []
    
    for annotator in qualified_annotators:
        
        df_qualified_annotators = df[df['user_id'] == annotator]
        for each_id in list(set(df_qualified_annotators['summary_uuid'].values)):
            df_uid = df_qualified_annotators[df_qualified_annotators['summary_uuid'] == each_id]
            if len(df_uid) == 1:
                row_append = df_uid.iloc[[0]]
            else:
                row_append = df_uid.iloc[[-1]]
            processed_rows.append(row_append)
            
    df_processed = pd.concat(processed_rows)
    return df_processed
                
        

                
    

In [64]:
df = remove_duplicates(df_annotations, qualified_annotators_billsum)
df.head()

,uuid,user_id,summary_uuid,summ_id,system_id,label_type,summary,nonfactual_sentences,article
66,67,ann_krcnbm,4f011fe6-8caf-4693-a64c-9caca556b5cc_gpt3_gen,billsum_generic,gpt3,non_factual,The Trade Dress Protection Act is a new law th...,"The act also amends the Trademark Act of 1946,...",SECTION 1. SHORT TITLE.\r\n\r\n This Act ma...
107,108,ann_krcnbm,b9eab77b-1667-472a-a44a-7bf03ae272cd_gpt3_gen,billsum_generic,gpt3,non_factual,The Tobacco Advertising and Promotion Studies ...,The act is based on Congress' findings that to...,SECTION 1. SHORT TITLE.\r\n\r\n This Act ma...
71,72,ann_krcnbm,6031839a-de71-4eed-8a08-1d7218747543_flant5_gen,billsum_generic,flant5,factual,To amend the Internal Revenue Code of 1986 to ...,,SECTION 1. SHORT TITLE.\r\n\r\n This Act ma...
108,109,ann_krcnbm,be653902-8770-480c-a0d9-b9f70081157c_flant5_gen,billsum_generic,flant5,factual,To amend the Internal Revenue Code of 1986 to ...,,SECTION 1. SHORT TITLE.\r\n\r\n This Act ma...
110,111,ann_krcnbm,be9b5d8c-35fd-4b30-9f87-02f759883304_flant5_gen,billsum_generic,flant5,non_factual,A bill to reduce the deficit by a total of $70...,A bill to reduce the deficit by a total of $70...,67 with respect to deficit reduction are \r\na...


In [68]:
df_flan = df[df['system_id'] == 'gpt3']
df_flan = df_flan[df_flan['label_type'] == 'non_factual']
nonfact_sents = 
len(set(df_flan['summary_uuid']))/len(df_flan)

0.6507936507936508

In [66]:
# print(df['article'].values[12])

In [35]:
''' Annotator agreement 
Same labels 
Same labels and sentences 
'''
user_ids = list(set(df['user_id'].values))
df_ann1 = df[df['user_id'] == user_ids[0]]
df_ann2 = df[df['user_id'] == user_ids[1]]

df_ann1_overlap = df_ann1[df_ann1['summary_uuid'].isin(df_ann2['summary_uuid'].values)]
df_ann2_overlap = df_ann2[df_ann2['summary_uuid'].isin(df_ann1['summary_uuid'].values)]
df_ann1_overlap = df_ann1_overlap.sort_values(['summary_uuid'])
df_ann2_overlap = df_ann2_overlap.sort_values(['summary_uuid'])
assert(list(df_ann1_overlap['summary_uuid'].values) == list(df_ann2_overlap['summary_uuid'].values))

print('Progress', len(df_ann2_overlap)/100)
label_aggr_condition = df_ann2_overlap['label_type'].values == df_ann1_overlap['label_type'].values
sentence_aggr_condition = df_ann2_overlap['nonfactual_sentences'].values == df_ann1_overlap['nonfactual_sentences'].values

print(' Aggreement (labels only) : ', \
      len(df_ann2_overlap[label_aggr_condition])/len(df_ann2_overlap))
print(' Aggreement (labels and sentences) : ', 
     len(df_ann2_overlap[(label_aggr_condition) & (sentence_aggr_condition) ])/len(df_ann2_overlap))


Progress 1.0
 Aggreement (labels only) :  0.82
 Aggreement (labels and sentences) :  0.82


In [36]:
# df_ann1

In [1]:
'''
Error Statistics:
Labeled as nonfactual --> absolute score (1/0)
Labeld as nonfactual --> get overlapping sentences --> get score of #nonfact sentences/len(summary sents)
Record relative scores and make a new df to get correlation
'''
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')






def make_error_stats(datatypes, qualified_annotators, agreement):
    non_factual_dict = {
                    'article': [],
                    'summary': [],
                    'model' : [],
                    'data': [],
                    'non_factual_sents': [], 
                    'label_type': [], \
                  'num_summary_sents' : []}
    for datatype in datatypes:
        df_data = df[df['summ_id'] == f'{datatype}_generic']
        
        for model in ['flant5', 'gpt3']:
            df_model = df_data[df_data['system_id'] == model]

            for unique_ids in list(set(df_model['summary_uuid'])):
                
                df_uuid = df_model[df_model['summary_uuid'] == unique_ids]
                # print(df_uuid)
                article = df_uuid['article'].values[0]
                user_ids_uuid =   set(list(df_uuid['user_id'].values))
                summary = df_uuid['summary'].values[0]
                summ_sents =  list(nlp(summary).sents)
                sentences_ann = list(df_uuid['nonfactual_sentences'].values)
                    
                annotated_sentences = []
                for each_sent in sentences_ann:
                    score = 0
                    if each_sent:
                        score = len(each_sent.split('<new_annotation>'))
                    annotated_sentences.append(score/len(summ_sents))
                
                sentbased_score = np.mean(annotated_sentences)
                # print(sentbased_score, annotated_sentences)
                non_factual_dict['article'] += [article]
                non_factual_dict['summary'] += [summary]
                non_factual_dict['model'] += [model]
                non_factual_dict['data'] += [datatype]
                non_factual_dict['non_factual_sents'] += [sentbased_score]
                label_type = 1 if sentbased_score >0 else 0
                non_factual_dict['label_type'] += [label_type]
                non_factual_dict['num_summary_sents'] += [len(summ_sents)]
                            
    return pd.DataFrame(non_factual_dict)

            
            
            
    
          


SyntaxError: invalid syntax (270690715.py, line 48)

In [39]:
# datatypes = ['xsum', 'cnndm']
# datatypes = ['billsum']
datatypes = ['pubmed']
qualified_annotators = qualified_annotators_pubmed
agreement = 2
df_error = make_error_stats(datatypes, qualified_annotators, agreement)
# df_error.to_csv(f'error_scores_{"_".join(datatypes)}.csv')
df_error

,article,summary,model,data,non_factual_sents,label_type,num_summary_sents
0,an important goal of microarray studies with ...,a comparison of the performance of classificat...,flant5,pubmed,0.000000,0,1
1,massively parallel sequencing technology can ...,a method for detecting copy-number variants fr...,flant5,pubmed,0.500000,1,1
2,mitochondrial genomes are physically separate...,This article describes the complete mitochondr...,flant5,pubmed,0.000000,0,1
3,it is well known that the teflon event involv...,pfoa is a persistent and non-biodegradable che...,flant5,pubmed,0.411765,1,17
4,a great deal of effort and expense are being ...,a java based linkage disequilibrium plotter,flant5,pubmed,0.000000,0,1
...,...,...,...,...,...,...,...
95,"feline herpesvirus, an alphaherpesvirus, is o...",Feline herpesvirus is a common virus in cats t...,gpt3,pubmed,0.000000,0,6
96,genome sequencing projects are generating mas...,This article discusses the challenges in annot...,gpt3,pubmed,0.000000,0,4
97,one of the critical considerations in cereal ...,The article discusses the importance of select...,gpt3,pubmed,0.000000,0,4
98,the ca2+-dependent activator protein for secr...,The article discusses the Ca2+-dependent activ...,gpt3,pubmed,0.100000,1,5


In [40]:
####### News
for data in list(set(df_error['data'])):
    print('Data' , data)
    for model in list(set(df_error['model'])):
        print('Model', model)
        df_error_dm = df_error[(df_error['data'] == data) & (df_error['model'] == model)]
        num_non_factual = list(df_error_dm['label_type'].values)
        num_non_factual_sents = list(df_error_dm['non_factual_sents'].values)
        num_summ_sents = list(df_error_dm['num_summary_sents'].values)
        if len(df_error_dm):
            print('Avg num sentences', np.mean(num_summ_sents))
            # print(sum(num_non_factual)/len(df_error_dm))
            print(np.mean(num_non_factual_sents))
            
        

Data pubmed
Model gpt3
Avg num sentences 5.46
0.028070818070818074
Model flant5
Avg num sentences 2.28
0.06115196078431372


In [26]:
for model in list(set(df_error['model'])):
        print('Model', model)
        df_error_dm = df_error[(df_error['model'] == model)]
        num_non_factual = list(df_error_dm['label_type'].values)
        num_non_factual_sents = list(df_error_dm['non_factual_sents'].values)
        num_summ_sents = list(df_error_dm['num_summary_sents'].values)
        if len(df_error_dm):
            print('Avg num sentences', np.mean(num_summ_sents))
            # print(sum(num_non_factual)/len(df_error_dm))
            print(np.mean(num_non_factual_sents))

Model gpt3
Avg num sentences 4.84
0.2008095238095238
Model flant5
Avg num sentences 1.0
0.23


In [82]:
df_gpt

,model,data,non_factual_sents,label_type,num_summary_sents


In [142]:
'''
Correlation of summary length with factuality score

'''

'\nCorrelation of summary length with factuality score\n\n'

In [145]:
import pandas as pd

def get_error_stats(df, data, qualified_annotators, agreement = 2):
    print('Fraction of factual errors')
    df_data = df[df['summ_id'] == f'{data}_generic']
    print(f'{data}_generic', len(df_data))
    for model in ['flant5', 'gpt3']:
        df_model = df_data[df_data['system_id'] == model]
        print(len(df_model))
        non_factual_num = []
        filtered_ids = []
        for unique_ids in list(set(df_model['summary_uuid'])):
            df_uuid = df_model[df_model['summary_uuid'] == unique_ids]
            user_ids_uuid =   set(list(df_uuid['user_id'].values))
            if len(user_ids_uuid) >= agreement:
                filtered_ids.append(unique_ids)
                label_types = list(df_uuid['label_type'].values)

                if len(set(label_types)) == 1:
                    if label_types[0] == 'non_factual':
                        non_factual_num.append(unique_ids)
        print(model, len(non_factual_num)/len(filtered_ids), len(filtered_ids))
            
    
          

In [146]:
get_error_stats(df, 'xsum', qualified_annotators_news)
get_error_stats(df, 'cnndm', qualified_annotators_news)

Fraction of factual errors
xsum_generic 98
49
flant5 0.5 24
49
gpt3 0.4583333333333333 24
Fraction of factual errors
cnndm_generic 45
22
flant5 0.0 7
23
gpt3 0.2857142857142857 7


In [63]:
def map_dicts(df):
    summary_annotations = {}
    label_annotations = {}
    sent_annotations = {}
    all_summ_uuids = sorted(list(set(df['summary_uuid'].values)), key = len)


    for idx, row in df.iterrows():
        user_id = row['user_id']
        summ_uuid = row['summary_uuid']
        label_type = row['label_type']
        nonfact_sents = row['nonfactual_sentences']

        if summ_uuid not in summary_annotations:
            summary_annotations[summ_uuid] = {}

        if user_id not in label_annotations:
            label_annotations[user_id] = {'factual': [], 'non_factual': []}
            sent_annotations[user_id] = [''] * len(all_summ_uuids)

        if label_type not in summary_annotations[summ_uuid]:
            summary_annotations[summ_uuid][label_type] = []

        summary_annotations[summ_uuid][label_type].append((user_id, nonfact_sents))

        label_annotations[user_id][label_type].append(summ_uuid)
    #     print(summ_uuid, all_summ_uuids.index(summ_uuid))
        sent_annotations[user_id][all_summ_uuids.index(summ_uuid)] = nonfact_sents
    
    return summary_annotations, label_annotations, sent_annotations

In [64]:
def compute_annotator_matrix_label(label_annotations):
    all_ann_ids = sorted(list(label_annotations.keys()), key = len)
    ann_matrix = {}
    ann_matrix['annotators'] = all_ann_ids
    for ann_id in all_ann_ids:
        if ann_id not in ann_matrix:
            ann_matrix[ann_id] = []
        ann_id_len = len(label_annotations[ann_id]['non_factual']) 
        for idx, other_ann_id in enumerate(all_ann_ids):
            num_factual_aggr = set(label_annotations[ann_id]['non_factual']).intersection(label_annotations[other_ann_id]['non_factual'])
            # num_nonfactual_aggr = set(label_annotations[ann_id]['factual']).intersection(label_annotations[other_ann_id]['factual'])
            ann_matrix[ann_id].append(len(num_factual_aggr)/ann_id_len)
    return ann_matrix

In [72]:
import numpy as np
def compute_annotator_matrix_sent(sent_annotations):
    all_ann_ids = sorted(list(label_annotations.keys()), key = len)
    ann_matrix = {}
    ann_matrix['annotators'] = all_ann_ids
    for ann_id in all_ann_ids:
        if ann_id not in ann_matrix:
            ann_matrix[ann_id] = []
            
        # ann_sents = sent_annotations[ann_id]
        
        for other_ann_id in all_ann_ids:
            other_ann_sent_counter = 0
            # other_ann_sents = sent_annotations[ann_id]
            # other_ann_sents = other_ann_sents.split('<new_annotation>')
            other_ann_sent_counter = []
            for idx, ann_sent in enumerate(sent_annotations[ann_id]):
#                 print(idx)
                other_ann_sent = sent_annotations[other_ann_id][idx]
                ann_sents_idx = ann_sent.split('<new_annotation>')
                other_ann_sents_idx = other_ann_sent.split('<new_annotation>')
                
                val = len(set(ann_sents_idx).intersection(other_ann_sents_idx))/len(ann_sents_idx)
                val = round(val, 1)
                # print(val)
                # if ann_sent == other_ann_sent:
                #     val = 1
                # elif (ann_sent in other_ann_sent) or (other_ann_sent in ann_sent):
                #     val = 0.5
                # else:
                #     val = 0
                other_ann_sent_counter += [val]
            ann_matrix[ann_id].append(np.mean(other_ann_sent_counter))
    return ann_matrix

In [73]:
summary_annotations, label_annotations, sent_annotations = map_dicts(df)

In [74]:
# label_annotations

In [75]:
ann_matrix = compute_annotator_matrix_label(label_annotations)
df_matrix = pd.DataFrame(ann_matrix)
df_matrix

,annotators,ann_japq,ann_tpfo
0,ann_japq,1.00000,0.675676
1,ann_tpfo,0.78125,1.000000


In [76]:
sent_matrix = compute_annotator_matrix_sent(sent_annotations)
df_sent_matrix = pd.DataFrame(sent_matrix)
df_sent_matrix

,annotators,ann_japq,ann_tpfo
0,ann_japq,1.000000,0.738272
1,ann_tpfo,0.749383,1.000000


In [74]:
len(label_annotations['ann_krcnbm']['non_factual'])

32